In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append('../')

In [3]:
from src.utils import loading, Spark
import pyspark.ml as M
import pyspark.sql.functions as F
import pyspark.sql.types as T
from pyspark.ml.recommendation import ALS, ALSModel
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.tuning import ParamGridBuilder, TrainValidationSplit
from tqdm.notebook import tqdm

In [4]:
from src import indexTransformer
from scipy import sparse
from sklearn import metrics
from sklearn.preprocessing import MinMaxScaler

In [7]:
import pyspark.sql.window as W

In [5]:
spark = Spark()

Spark UI address http://127.0.0.1:4040


In [52]:
ratings = loading(spark, '../data/raw')['sample']

In [53]:
window = W.Window.partitionBy(ratings['userId']).orderBy(ratings['timestamp'].desc())

In [60]:
ranked = ratings.select('*', F.rank().over(window).alias('rank'))

In [61]:
rating_count = ratings.groupby('userId').agg(F.count('movieId').alias('cnt'))

In [62]:
ranked = ranked.join(rating_count, ranked.userId == rating_count.userId)\
        .select(ranked.userId, ranked.movieId, ranked.timestamp, ranked.rating, ranked.rank, rating_count.cnt)

In [63]:
ranked = ranked.withColumn('position', 1 - F.col('rank')/F.col('cnt'))\
        .select('userId', 'movieId', 'timestamp','rating', 'position')

In [64]:
train = ranked.where(ranked.position < .75)
test = ranked.where(ranked.position >= .75)

In [65]:
train.count()

339624

In [66]:
test.count()

104665